# Spectrum Exctraction Quickstart

Specreduce provides a flexible toolset for extracting a 1D spectrum from a
2D spectral image, including steps for determining the trace of a spectrum,
background subtraction, extraction, and wavelength calibration. 

This quickstart covers the basic spectrum extraction and calibration steps for
a synthetic science spectrum observation accompanied by He, Ne, and Ar arc-lamp
observations for wavelength calibration.

This quickstart tutorial is written as a Jupyter Notebook, and you can run it 
easily yourself by either copying it (and the `common.py` module) from GitHub,
or from your own specreduce installation under the `docs/getting_started` directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from astropy.modeling import models
from astropy.nddata import StdDevUncertainty
from astropy.stats import mad_std

from specreduce.tracing import FlatTrace, FitTrace
from specreduce.extract import BoxcarExtract, HorneExtract
from specreduce.background import Background
from specreduce.wavecal1d import WavelengthCalibration1D

In [ ]:
from common import make_science_and_arcs, plot_2d_spectrum

## Data preparation

First, we start by "reading in" our science and arc spectra. We use synthethic data 
created by the `common.make_science_and_arcs` utility function, which itself uses 
`specreduce.utils.synth_data.make_2d_arc_image` and `specreduce.utils.synth_data.make_2d_spec_image` 
functions. The science frame (`sci`) and the three arc frames (`arc_he`, `arc_ne`, 
and `arc_ar`) are created as `astropy.nddata.CCDData` objects with `astropy.nddata.StdDevUncertainty` 
uncertainties.

In [ ]:
sci, (arc_he, arc_ne, arc_ar) = make_science_and_arcs(1000, 300)
arcs = arc_he, arc_ne, arc_ar

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(6, 6), constrained_layout=True, sharex='all')
plot_2d_spectrum(sci, ax=axs[0], label='Science')
for i, (arc, label) in enumerate(zip(arcs, "HeI NeI ArI".split())):
    plot_2d_spectrum(arc, ax=axs[i+1], label=label)

plt.setp(axs[:-1], xlabel='')
plt.setp(axs, ylabel='C.D. axis [pix]');

## Spectrum Tracing

The `specreduce.tracing` module defines the trace of a spectrum on the 2D image.
These traces can either be determined semi-automatically or manually, and are
provided as the inputs for the remaining steps of the extraction process.
Supported trace types include:

* `specreduce.tracing.ArrayTrace`
* `specreduce.tracing.FlatTrace`
* `specreduce.tracing.FitTrace`

Each of these trace classes takes the 2D spectral image as input, as well as
additional information needed to define or determine the trace (see the API docs
above for required parameters for each of the available trace classes)

If our spectrum would be flat along the cross-dispersion axis, we could use `FlatTrace`. 
However, our spectrum has quite significant curvature, and we need to use `FitTtrace`
that fits an `Astropy` model to the spectrum. 

In our case, we decide to use a third degree polynomial, and to bin the spectrum to 
20 bins along the dispersion axis. The `FitTrace` routine estimates the PSF 
centroid along the cross-dispersion axis for each bin, and then fits the given 
trace model to the centroids. The binning along the dispersion axis helps to
stabilise the fit that could otherwise be adversely affected by noise, since the
spectrum is faint.

In [ ]:
trace = FitTrace(sci, bins=40, guess=200, trace_model=models.Polynomial1D(3))

In [ ]:
fig, ax = plot_2d_spectrum(sci)
ax.plot(trace.trace, 'k', ls='--');

## Background Subtraction

The `specreduce.background` module contains tools to generate and subtract a 
background image from the input 2D spectral image.  The `specreduce.background.Background` 
object is defined by one or more windows, and can be generated with:

* `specreduce.background.Background.one_sided`
* `specreduce.background.Background.two_sided`
 
The center of the window can either be passed as a float/integer or as a trace

```Python
bg = specreduce.background.Background.one_sided(sci, trace, separation=5, width=2)
```

or, equivalently

```Python
bg = specreduce.background.Background.one_sided(sci, 15, separation=5, width=2)
```

The estimated background image can be accessed via `specreduce.background.Background.bkg_image`
and the background-subtracted image via `specreduce.background.Background.sub_image`.

Let's calculate and remove a two-sided background estimate.

In [ ]:
background = Background.two_sided(sci, trace, separation=50, width=30)

In [ ]:
fig, ax = plot_2d_spectrum(sci)
ax.plot(trace.trace, 'k--')
for bkt in background.traces:
    ax.fill_between(np.arange(sci.shape[1]), 
                    bkt.trace-background.width/2, 
                    bkt.trace+background.width/2, 
                    fc='w', alpha=0.2)
    ax.plot(bkt.trace+background.width/2, 'w--', lw=1)
    ax.plot(bkt.trace-background.width/2, 'w--', lw=1)

In [ ]:
sci_clean = background.sub_image()

In [ ]:
fig, ax = plot_2d_spectrum(sci_clean.flux.value)

## Spectrum Extraction

The `specreduce.extract` module extracts a 1D spectrum from an input 2D spectrum
(likely a background-extracted spectrum from the previous step) and a defined
window, using one of the following implemented methods:

* `specreduce.extract.BoxcarExtract`
* `specreduce.extract.HorneExtract`

The methods take the input image and trace as inputs, an optional mask treatment
agrumtn, and a set of method-specific arguments fine-tuning the behavior of each
method.

### Boxcar Extraction

Boxcar extraction requires a 2D spectrum, a trace, and the extraction aperture width.

In [ ]:
aperture_width = 20
e = BoxcarExtract(sci_clean, trace, width=aperture_width)
science_spectrum_boxcar = e.spectrum

The extracted spectrum can be accessed via the `spectrum` property or by calling
the `extract` object, which also allows you to override the values the object
was initialized with.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 4), constrained_layout=True, sharex='all')
plot_2d_spectrum(sci_clean.flux.value, ax=axs[0])
axs[0].fill_between(np.arange(sci_clean.shape[1]), 
                    trace.trace-aperture_width/2, 
                    trace.trace+aperture_width/2, 
                    fc='w', alpha=0.25, ec='k', ls='--')

axs[1].plot(science_spectrum_boxcar.flux)
plt.setp(axs[1],
         xlabel=f'Wavelength [{science_spectrum_boxcar.spectral_axis.unit}]', 
         ylabel=f'Flux [{science_spectrum_boxcar.flux.unit}]');
fig.align_ylabels()

### Horne Extraction

For the Horne algorithm, the variance array is required. If the input image is
an `~astropy.nddata.NDData` object with ``image.uncertainty`` provided,
then this will be used. Otherwise, the ``variance`` parameter must be set.

```Python
extract = specreduce.extract.HorneExtract(image-bg, trace, variance=var_array)
```

The Horne algorithm provides two options for fitting the spatial profile to the
cross dispersion direction of the source: a Gaussian fit (default),
or an empirical ``interpolated_profile`` option.

If the default Gaussian option is used, an optional background model may be
supplied as well (default is a 2D Polynomial) to account
for residual background in the spatial profile. This option is not supported for
``interpolated_profile``.


If  the ``interpolated_profile`` option is used, the image will be sampled in various
wavelength bins (set by ``n_bins_interpolated_profile``), averaged in those bins, and
samples are then interpolated between (linear by default, interpolation degree can
be set with ``interp_degree_interpolated_profile``, which defaults to linear in
x and y) to generate an empirical interpolated spatial profile. Since this option
has two optional parameters to control the fit, the input can either be a string
to indicate that ``interpolated_profile`` should be used for the spatial profile
and to use the defaults for bins and interpolation degree, or to override these
defaults a dictionary can be passed in.

For example, to use the ``interpolated_profile`` option with default bins and
interpolation degree

    interp_profile_extraction = extract(spatial_profile='interpolated_profile')

Or, to override the default of 10 samples and use 20 samples

    interp_profile_extraction = extract(spatial_profile={'name': 'interpolated_profile',
                                    'n_bins_interpolated_profile': 20)

Or, to do a cubic interpolation instead of the default linear

    interp_profile_extraction = extract(spatial_profile={'name': 'interpolated_profile',
                                    'interp_degree_interpolated_profile': 3)

As usual, parameters can either be set when instantiating the HorneExtraxt object,
or supplied/overridden when calling the extraction method on that object.

In [ ]:
e = HorneExtract(sci_clean, trace)
science_spectrum_horne = e.spectrum

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 2), constrained_layout=True, sharex='all')
ax.plot(science_spectrum_horne.flux)
plt.setp(ax,
         xlabel=f'Wavelength [{science_spectrum.spectral_axis.unit}]', 
         ylabel=f'Flux [{science_spectrum.flux.unit}]')
ax.autoscale(axis='x', tight=True)

## Wavelength Calibration

In [ ]:
spec_he = BoxcarExtract(arc_he, trace, width=aperture_width).spectrum
spec_ne = BoxcarExtract(arc_ne, trace, width=aperture_width).spectrum
spec_ar = BoxcarExtract(arc_ar, trace, width=aperture_width).spectrum

In [ ]:
spec_he.uncertainty = StdDevUncertainty(np.sqrt(np.abs(spec_he.flux.value)))
spec_ne.uncertainty = StdDevUncertainty(np.sqrt(np.abs(spec_ne.flux.value)))
spec_ar.uncertainty = StdDevUncertainty(np.sqrt(np.abs(spec_ar.flux.value)))

In [ ]:
wc = WavelengthCalibration1D([spec_he, spec_ne, spec_ar], line_lists=["HeI", "NeI", "ArI"], line_list_bounds=(4900, 10_000))

In [ ]:
import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    wc.find_lines(3, noise_factor=20)

In [ ]:
fig = wc.plot_fit(figsize=(6, 10))
plt.setp(fig.axes[::2], xlim=(4900, 10_000));

In [ ]:
ws = wc.fit_lines([72, 295, 403, 772], [5877, 6680, 7067, 8380], degree=3)

In [ ]:
fig = wc.plot_fit(figsize=(6, 10), obs_to_wav=True)
plt.setp(fig.axes, xlim=ws.p2w([0, 1000]));

In [ ]:
wc.plot_residuals();

In [ ]:
science_spectrum_resampled = ws.resample(science_spectrum_boxcar)

In [ ]:
science_spectrum_resampled.plot()